In [1]:
import pandas as pd
import warnings
import numpy as np
import tensorflow as tf
import random

# Фиксация random seed для воспроизводимости
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)


# Игнорировать все предупреждения
warnings.filterwarnings('ignore')

# Загрузка данных
file_path = "Meteo1-2023-15min(resampled).csv"
df = pd.read_csv(file_path, delimiter=",")

# Преобразование времени в datetime и установка индекса
df["time_YYMMDD_HHMMSS"] = pd.to_datetime(
    df["time_YYMMDD_HHMMSS"], format="%Y-%m-%d %H:%M:%S"
)
df.set_index("time_YYMMDD_HHMMSS", inplace=True)

# Добавление новых признаков
df["month"] = df.index.month
df["day_of_month"] = df.index.day
df["hour"] = df.index.hour

# Просмотр первых строк датафрейма с новыми признаками
print(df.head())

                     WindSpeedMax  WindDirection  AirTemperature  AirPressure  \
time_YYMMDD_HHMMSS                                                              
2023-01-01 00:00:00      5.433333     292.256367        4.926154  1028.344333   
2023-01-01 00:15:00      6.100000     293.810423        4.976190  1028.433600   
2023-01-01 00:30:00      5.200000     303.513670        4.957143  1028.502367   
2023-01-01 00:45:00      5.700000     297.176083        4.835715  1028.429633   
2023-01-01 01:00:00      6.000000     295.580110        4.493651  1028.400100   

                     AirHumidity  WindSpeed  month  day_of_month  hour  
time_YYMMDD_HHMMSS                                                      
2023-01-01 00:00:00    88.889567   4.422906      1             1     0  
2023-01-01 00:15:00    88.600962   4.739206      1             1     0  
2023-01-01 00:30:00    88.833338   4.109524      1             1     0  
2023-01-01 00:45:00    90.213968   4.430159      1             1   

In [2]:
from utils import cyclical_feature_encoding

df_transformed = cyclical_feature_encoding(
    df, ["WindDirection", "month", "day_of_month", "hour"]
)
print(df_transformed.head())
print(df_transformed.tail())

                     WindSpeedMax  AirTemperature  AirPressure  AirHumidity  \
time_YYMMDD_HHMMSS                                                            
2023-01-01 00:00:00      5.433333        4.926154  1028.344333    88.889567   
2023-01-01 00:15:00      6.100000        4.976190  1028.433600    88.600962   
2023-01-01 00:30:00      5.200000        4.957143  1028.502367    88.833338   
2023-01-01 00:45:00      5.700000        4.835715  1028.429633    90.213968   
2023-01-01 01:00:00      6.000000        4.493651  1028.400100    89.865232   

                     WindSpeed  WindDirection_sin  WindDirection_cos  \
time_YYMMDD_HHMMSS                                                     
2023-01-01 00:00:00   4.422906          -0.920757           0.390137   
2023-01-01 00:15:00   4.739206          -0.909811           0.415024   
2023-01-01 00:30:00   4.109524          -0.826614           0.562769   
2023-01-01 00:45:00   4.430159          -0.883810           0.467847   
2023-01-01 01:

In [3]:
from utils import generate_time_series_features, calculate_horizons

# Используем параметры, которые дали лучший результат
lookback_hours = 12
forecast_hours = 3
lag_step = 4
rolling_window_min = 3
rolling_window_max = 15
expanding_window_min = 3
expanding_window_max = 15

target_variable = "WindSpeed"

# Вычисление горизонта прогноза и лагов
forecast_horizon, lookback_horizon = calculate_horizons(
    df_transformed, forecast_hours, lookback_hours
)

lags = list(range(1, lookback_horizon, lag_step))
rolling_window_sizes = list(
    range(rolling_window_min, min(rolling_window_max, lookback_horizon), lag_step)
)
expanding_window_sizes = list(
    range(expanding_window_min, min(expanding_window_max, lookback_horizon), lag_step)
)

# Генерация признаков
df_transformed = generate_time_series_features(
    df_transformed,
    columns=[
        "WindSpeedMax",
        "AirTemperature",
        "AirPressure",
        "AirHumidity",
        "WindSpeed",
        "WindDirection_sin",
        "WindDirection_cos",
    ],
    lags=lags,
    rolling_window_sizes=rolling_window_sizes,
    expanding_window_sizes=expanding_window_sizes,
)

# Display the transformed DataFrame
df_transformed.head()


,WindSpeedMax,AirTemperature,AirPressure,AirHumidity,WindSpeed,WindDirection_sin,WindDirection_cos,month_sin,month_cos,day_of_month_sin,...,WindDirection_cos_lag_13,WindDirection_cos_lag_9,WindDirection_cos_lag_5,WindDirection_cos_lag_1,WindDirection_cos_rolling_mean_3,WindDirection_cos_rolling_mean_7,WindDirection_cos_rolling_mean_11,WindDirection_cos_cum_mean_3,WindDirection_cos_cum_mean_7,WindDirection_cos_cum_mean_11
time_YYMMDD_HHMMSS,,,,,,,,,,,,,,,,,,,,,
2023-01-01 11:15:00,2.433333,14.167777,1027.883767,57.008889,1.849206,-0.350623,-0.936517,0.5,0.866025,0.201299,...,0.370203,0.353096,0.359597,-0.677961,-0.605172,-0.163317,0.016782,0.331054,0.331054,0.331054
2023-01-01 11:30:00,2.433333,14.483967,1027.890567,54.828733,1.722540,0.183026,-0.983108,0.5,0.866025,0.201299,...,0.350329,0.389428,0.329963,-0.936517,-0.865862,-0.337350,-0.110719,0.303093,0.303093,0.303093
2023-01-01 11:45:00,2.900000,14.368413,1027.786600,56.345559,1.866508,0.736402,-0.676544,0.5,0.866025,0.201299,...,0.354931,0.165897,-0.252388,-0.983108,-0.865390,-0.485370,-0.204323,0.282684,0.282684,0.282684
2023-01-01 12:00:00,2.200000,12.847460,1027.770767,63.596039,1.457460,0.988295,0.152556,0.5,0.866025,0.201299,...,0.419405,0.235122,-0.201038,-0.676544,-0.502365,-0.510714,-0.225856,0.280028,0.280028,0.280028
2023-01-01 12:15:00,3.466667,11.488889,1027.700100,73.411745,2.969841,0.652139,0.758100,0.5,0.866025,0.201299,...,0.353096,0.359597,-0.677961,0.152556,0.078037,-0.366359,-0.172020,0.289590,0.289590,0.289590


In [4]:
from sklearn.model_selection import train_test_split
from utils import prepare_ml_data_from_features

# Разделение данных на обучающую и тестовую выборки
df_train, df_test = train_test_split(df_transformed, test_size=0.1, random_state=42, shuffle=False)

# Подготовка данных для машинного обучения
X_train, y_train = prepare_ml_data_from_features(df_train, target_variable=target_variable, forecast_horizon=forecast_horizon)
X_test, y_test = prepare_ml_data_from_features(df_test, target_variable=target_variable, forecast_horizon=forecast_horizon)

# Вывод размеров данных
print(f"Размеры обучающего набора X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размеры тестового набора X_test: {X_test.shape}, y_test: {y_test.shape}")

Размеры обучающего набора X_train: (31484, 139), y_train: (31484, 12)
Размеры тестового набора X_test: (3488, 139), y_test: (3488, 12)


In [5]:
from sklearn.model_selection import train_test_split
from utils import prepare_ml_data_from_features

# Разделение данных на обучающую и тестовую выборки
df_train, df_test = train_test_split(df_transformed, test_size=0.1, random_state=42, shuffle=False)

# Подготовка данных для машинного обучения
X_train, y_train = prepare_ml_data_from_features(df_train, target_variable=target_variable, forecast_horizon=forecast_horizon)
X_test, y_test = prepare_ml_data_from_features(df_test, target_variable=target_variable, forecast_horizon=forecast_horizon)

# Вывод размеров данных
print(f"Размеры обучающего набора X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Размеры тестового набора X_test: {X_test.shape}, y_test: {y_test.shape}")


Размеры обучающего набора X_train: (31484, 139), y_train: (31484, 12)
Размеры тестового набора X_test: (3488, 139), y_test: (3488, 12)


In [6]:
import plotly.graph_objs as go
import plotly.subplots as sp
import numpy as np

def plot_sample_with_features(X, y, target_variable, columns, y_pred_values=None):
    """
    Функция для визуализации одного случайного примера из данных X и y с использованием Plotly.
    
    :param X: Матрица признаков (включая лаги целевой переменной)
    :param y: Целевая переменная (например, ветер на 12 шагов вперед)
    :param target_variable: Название целевой переменной
    :param columns: Список всех колонок, которые будут визуализироваться
    :param y_pred_values: Прогнозируемые значения (если есть)
    """
    # Выбор случайного индекса
    idx = np.random.randint(0, len(X))

    # Фильтрация лагов для каждой колонки в columns и сортировка по убыванию
    lagged_features = {col: [col_name for col_name in X.columns if f"{col}_lag" in col_name] for col in columns}

    # Создание субплотов
    num_features = len(columns)
    fig = sp.make_subplots(rows=num_features, cols=1, shared_xaxes=False, vertical_spacing=0.05)

    for i, column in enumerate(columns):
        # Данные по лагам для текущей колонки и текущее значение
        x_lags = X[lagged_features[column] + [column]].iloc[idx].values

        # Добавление данных на график для текущего признака
        fig.add_trace(go.Scatter(x=list(range(len(x_lags))), y=x_lags,
                                 mode='lines+markers', name=f'{column} (lags and current)',
                                 line=dict(color='green'), showlegend=(i == 0)), row=i+1, col=1)

        if column == target_variable:
            # Значения y для прогноза, только для целевой переменной
            y_values = y[idx]

            fig.add_trace(go.Scatter(x=list(range(len(x_lags), len(x_lags) + len(y_values))), y=y_values,
                                     mode='lines+markers', name=f'{column} (Future)',
                                     line=dict(color='red'), showlegend=(i == 0)), row=i+1, col=1)

            if y_pred_values is not None:
                fig.add_trace(go.Scatter(x=list(range(len(x_lags), len(x_lags) + len(y_pred_values[idx]))), y=y_pred_values[idx],
                                         mode='lines+markers', name=f'{column} (Predicted)',
                                         line=dict(color='blue'), showlegend=(i == 0)), row=i+1, col=1)

        # Вертикальная линия для разделения лагов и прогнозов
        fig.add_vline(x=len(x_lags)-1, line=dict(color='gray', dash='dash'), row=i+1, col=1)

    # Обновление разметки и отображение графика
    fig.update_layout(height=num_features * 400, width=1500, title_text="Sample Visualization with Features")
    fig.show()

# Пример использования
plot_sample_with_features(X_train, y_train, target_variable='WindSpeed', columns=[
        "WindSpeedMax",
        "AirTemperature",
        "AirPressure",
        "AirHumidity",
        "WindSpeed",
        "WindDirection_sin",
        "WindDirection_cos",
    ])

In [7]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from utils import calculate_metrics


# Масштабирование данных X
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Масштабирование данных y
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Формат данных
input_shape = (X_train_scaled.shape[1],)  # Входной вектор признаков
forecast_horizon = y_train_scaled.shape[1]  # Количество шагов вперед

# Создание модели с оптимальными параметрами
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=input_shape))  # Уменьшение нейронов на основе оптимального результата
model.add(Dense(100, activation='relu'))  # Оптимальные 5 слоев
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))  # Оптимальный последний слой с 50 нейронами
model.add(Dense(forecast_horizon))  # Прогнозируем N шагов вперед

# Компиляция модели с заданным learning rate
model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse')

# Настройка коллбека для сохранения лучших весов модели по валидационной потере
checkpoint = ModelCheckpoint('best_model.weights.h5', monitor='val_loss', save_best_only=True, mode='min', save_weights_only=True)

# Настройка коллбека для динамического изменения learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Обучение модели с использованием reduce_lr
model.fit(X_train_scaled, y_train_scaled, epochs=150, batch_size=256, validation_split=0.2, callbacks=[checkpoint, reduce_lr])

# Загрузка лучших сохраненных весов
model.load_weights('best_model.weights.h5')

# Прогнозирование
y_pred_scaled = model.predict(X_test_scaled)

# Обратное масштабирование y_pred
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Вывод метрик
print(calculate_metrics(y_test, y_pred, y_train))

Epoch 1/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.9780 - val_loss: 0.9397 - learning_rate: 5.0000e-05
Epoch 2/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7436 - val_loss: 0.6172 - learning_rate: 5.0000e-05
Epoch 3/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5282 - val_loss: 0.5006 - learning_rate: 5.0000e-05
Epoch 4/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4568 - val_loss: 0.4793 - learning_rate: 5.0000e-05
Epoch 5/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4328 - val_loss: 0.4742 - learning_rate: 5.0000e-05
Epoch 6/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4187 - val_loss: 0.4732 - learning_rate: 5.0000e-05
Epoch 7/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4084 - val_loss: 0.4740 - learning_rate: 5.0000e-05
Epoch 8/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3999 - val_loss: 0.4756 - learning_rate: 5.0000e-05
Epoch 9/150
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3927 - val_loss: 0.4776 - learning_

In [8]:
y_pred.shape

(3488, 12)

In [9]:
plot_sample_with_features(X_test, y_test, target_variable='WindSpeed', columns=[
        "WindSpeed"], y_pred_values = y_pred
    )

In [10]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test[:, 0]))),
    y=y_test[:, 0],
    mode='lines',
    name='Actual',
    line=dict(color='blue')
))

# Add predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_pred[:, 0]))),
    y=y_pred[:, 0],
    mode='lines',
    name='Predicted',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values',
    xaxis_title='Time Steps',
    yaxis_title='Value',
    legend=dict(x=0, y=1),
    width=1800,
    height=600,
    showlegend=True
)

# Show the plot
fig.show()


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot figure with 12 rows and 1 column
fig = make_subplots(rows=12, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Iterate over each forecast step and add a trace for actual and predicted values
for i in range(12):
    fig.add_trace(go.Scatter(
        x=list(range(len(y_test[:, i]))),
        y=y_test[:, i],
        mode='lines',
        name=f'Actual (Step {i+1})',
        line=dict(color='blue')
    ), row=i+1, col=1)
    
    fig.add_trace(go.Scatter(
        x=list(range(len(y_pred[:, i]))),
        y=y_pred[:, i],
        mode='lines',
        name=f'Predicted (Step {i+1})',
        line=dict(color='red')
    ), row=i+1, col=1)

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values for Each Forecast Step',
    height=3000,  # Increase height to fit all subplots
    showlegend=False  # Disable legend to avoid repetition
)

# Show the plot
fig.show()


In [12]:
from utils import stitch_forecasts

# Применяем функцию к y_pred и y_test
y_pred_stitched = stitch_forecasts(y_pred)
y_test_stitched = stitch_forecasts(y_test)

y_pred_stitched.shape

(3492,)

In [14]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test_stitched))),
    y=y_test_stitched,
    mode='lines',
    name='Actual',
    line=dict(color='blue')
))

# Add predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_pred_stitched))),
    y=y_pred_stitched,
    mode='lines',
    name='Predicted',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title='Comparison of Predicted vs Actual Values',
    xaxis_title='Time Steps',
    yaxis_title='Value',
    legend=dict(x=0, y=1),
    width=1800,
    height=600,
    showlegend=True
)

# Show the plot
fig.show()
